#### Transform Customers Data

#### 1. Remove null customer id records

In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not null

#### 02.Identify duplicate and remove exact duplicate records

In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not null
order by customer_id

In [0]:
select distinct * 
from gizmobox.bronze.v_customers
where customer_id is not null
order by customer_id

#### 03.Create Temporary View

In [0]:
create temporary view t_v_distinct_customers
as
select distinct * 
from gizmobox.bronze.v_customers
where customer_id is not null
order by customer_id

In [0]:
with cte as (
  select customer_id,max(created_timestamp) as crtd_timestamp
  from t_v_distinct_customers
  where customer_id is not null
  group by customer_id
)
select cast(c.created_timestamp as Timestamp)
      ,c.customer_id
      ,c.customer_name
      ,cast(c.date_of_birth as date)
      ,c.email
      ,cast(c.member_since as date)
      ,c.telephone
from t_v_distinct_customers c join cte ct on c.customer_id = ct.customer_id and c.created_timestamp = ct.crtd_timestamp
where c.customer_id is not null
order by c.customer_id

#### 04.Create table in sliver schema

In [0]:
create table gizmobox.silver.customers
as
with cte as (
  select customer_id,max(created_timestamp) as crtd_timestamp
  from t_v_distinct_customers
  where customer_id is not null
  group by customer_id
)
select cast(c.created_timestamp as Timestamp)
      ,c.customer_id
      ,c.customer_name
      ,cast(c.date_of_birth as date)
      ,c.email
      ,cast(c.member_since as date)
      ,c.telephone
from t_v_distinct_customers c join cte ct on c.customer_id = ct.customer_id and c.created_timestamp = ct.crtd_timestamp
where c.customer_id is not null
order by c.customer_id


In [0]:
select * from gizmobox.silver.customers